In [7]:
import numpy as np
from numba import njit, vectorize
from BPnumba.GeneticOperators import CrossOX
from numba.typed import List as NumbaList


In [21]:


from BPnumba.GeneticOperators import RI, RIS, RRIS, RS, RSS


chromosome = NumbaList([1,2,3,4,5,6,7,8,9,10])
chromosomeR = NumbaList([0,0,0,0,0,0,0,0,0,0])
chromosome2 = NumbaList([2,8,5,1,9,6,3,4,7,10])
chromosomeR = NumbaList([0,0,0,0,0,0,0,0,0,0])

RSS(chromosome2,2,4, 6,8)

TypeError: not enough arguments: expected 5, got 3